# Titanic ML Competition

## Introduction

In this notebook we will be exploring the data for the Titanic machine learning competition from [Kaggle](https://www.kaggle.com/c/titanic/overview). The goals of the notebook are to:
* Better understand the data.
* See data relationships.
* Determine if there are patterns within the data.
* See what sorts of people were more likely to survive the disaster.

After this, we will create ML models to make predictions.

*Introduction from Kaggle*
```
The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).
```

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import squarify

In [ ]:
from math import pi

## Importing the dataset

In [ ]:
titanic_df = pd.read_csv("Datasets/train.csv")
titanic_df.head()

In [ ]:
titanic_df.info()

From the table above we can see the different data types that pandas assigned each column and that we have some columns with missing values. Before starting to work with these, we have to clean the data. 

In [ ]:
titanic_df.isnull().sum()

### Renaming of the columns & changing index

Here we change some of the names of the columns that doesn't sound that meaningful. (For instance SibSp or Parch). Also we make the index the passenger id, as it makes more sense there.

In [ ]:
renamed_columns = {"Pclass":"Economic status","SibSp":"Number of siblings/spouses","Parch":"Number of parents/children"}
titanic_df.rename(columns=renamed_columns,inplace = True)

In [ ]:
titanic_df.set_index(titanic_df["PassengerId"],inplace = True)
titanic_df.drop(columns="PassengerId",inplace=True)

### Data types

Now we will convert some columns to more appropriate data type, which will make things easier to work later. Additionally, this reduces the memory usage of the dataset.

The columns with object data type are candidates to be of categorical type. For this we check the cardinality they have.

In [ ]:
titanic_df.select_dtypes(include=['object']).nunique()

In [ ]:
titanic_df["Sex"] = titanic_df["Sex"].astype("category")
titanic_df["Embarked"] = titanic_df["Embarked"].astype("category")

Other column that could be categorical is the one representing the economic status.

In [ ]:
titanic_df["Economic status"] = titanic_df["Economic status"].astype("category")

Not always is about converting columns to categorical data types, we can also convert numerical types that use 64 bits to smaller sizes (such as 8 bits, 16, 32). By doing this we can reduce even further the memory usage of the dataset.

Before converting SibSp/Parch to a an integer of smaller size, we have to check the maximum number they have. *(Max int of int8:127(signed))*

In [ ]:
titanic_df[["Number of siblings/spouses","Number of parents/children"]].max()

As there is no problem we can make the conversions.

In [ ]:
titanic_df["Number of siblings/spouses"] = titanic_df["Number of siblings/spouses"].astype("int8")
titanic_df["Number of parents/children"] = titanic_df["Number of parents/children"].astype("int8")

In [ ]:
titanic_df["Survived"] = titanic_df["Survived"].astype("int8")

### Missing values

This is one of the most important things to do in data analysis. It will be mainly develop on the different preprocessings to the data for the models. Now we are just going to remove the `ticket` column because it doesn't seem like it will be useful, plus we will be filling the missings from the `cabin` column with `-`.

In [ ]:
titanic_df["Cabin"].fillna("-",inplace=True)

In [ ]:
titanic_df.drop(columns=["Ticket"],inplace=True)

### Data conversions

Here we change some of the texts the data has, so that they are more meaningful.

In [ ]:
new_economic_status_names = {1:"Upper",2:"Middle",3:"Lower"}
titanic_df["Economic status"].cat.rename_categories(new_economic_status_names,inplace=True)

In [ ]:
new_port_names = {"C":"Cherbourg","Q":"Queenstown","S":"Southhampton"}
titanic_df["Embarked"].cat.rename_categories(new_port_names,inplace = True)

In [ ]:
new_sex = {"male":"Male","female":"Female"}
titanic_df["Sex"].cat.rename_categories(new_sex,inplace = True)

### Reordering columns

Finally, we order the dataset in a more relevant way.

In [ ]:
personal_info = ["Name","Sex","Age"]
economic_status = ["Economic status","Fare"]
family = ["Number of siblings/spouses","Number of parents/children"]
journey = ["Cabin","Embarked","Survived"]
new_order = personal_info + economic_status + family + journey
titanic_df = titanic_df.reindex(columns = new_order)

### Dataset after handling it

In [ ]:
titanic_df.info()

Here we can see that the memory usage was reduced by approximately ~40KB.

In [ ]:
titanic_df.head()

## Exploratory Data Analysis

First, let's start by asking some simple questions that will get us closer to the question that matters. What sorts of people were more likely to survive?
* How many survived?
* How much does the sex determine the chances of survival?
* What about the age?
* Does the economic status helps to determine it?

### How many survived the disaster?

In [ ]:
survival_values = titanic_df['Survived'].value_counts()
names = ['Died',"Survived"]
plt.figure(figsize=(10, 5), dpi=100)
 
plt.subplot2grid(shape=(1,2),loc=(0,0))
plt.bar(x=survival_values.index,height=survival_values.values,color=['lightcoral', 'lightgreen'])
plt.xticks(survival_values.index,names)
plt.title("Amount of people that survived")

plt.subplot2grid(shape=(1,2),loc=(0,1))
plt.pie(survival_values, labels=names,colors=['lightcoral', 'lightgreen'], autopct='%1.0f%%')
plt.title("Proportion of people that survived")

plt.suptitle('Survival numbers')
plt.show()

We can see that approximately 40% of the passengers survived.

### How much does the sex affect the chances of survival?

Let's begin by seeing the proportions of the passengers.

In [ ]:
sex_proportions = titanic_df["Sex"].value_counts()
circle=plt.Circle( (0,0), 0.7, color='white')
plt.figure(dpi=80)
plt.pie(sex_proportions.values, labels=sex_proportions.index, colors=['goldenrod','salmon'],autopct='%1.0f%%')
p=plt.gcf()
p.gca().add_artist(circle)
plt.suptitle('Proportion of passengers')
plt.show()

Now let's see the survival rate.

In [ ]:
survival_by_sex = titanic_df.groupby(by="Sex")["Survived"].agg("mean")
survival_by_sex

In [ ]:
plt.figure(figsize=(4, 4), dpi=100)
plt.bar(x=survival_by_sex.index,height=survival_by_sex.values,color=['palevioletred', 'cadetblue'])
plt.ylim(top=1)
plt.title("Proportions of people that survived by sex")

We can clearly see that women proportionally had a greater survival rate than men.

### What about the age?

Can we see a pattern by exploring the age? Let's try to see if the children were more likely to survive.

In [ ]:
sns.distplot( a=titanic_df["Age"], hist=True, kde=False, rug=False )
plt.title("Age distribution")
plt.show()

We can see that the main group of people that was on the ship were adults, followed by children and then by old people.

In [ ]:
sns.violinplot(x=titanic_df["Survived"], y=titanic_df["Age"])

plt.title("Age distribution") 
plt.show()

Here we can see that children were more likely to survive by looking at the start of it.

In [ ]:
children = titanic_df["Age"] < 13
survived = titanic_df["Survived"] == 1
survived_age = titanic_df.loc[survived & children,"Age"]
died_age = titanic_df.loc[(~survived) & children,"Age"]

In [ ]:
survived_age.count()

In [ ]:
died_age.count()

### Does the economic status helps to determine it?

In [ ]:
total_class_members = titanic_df["Economic status"].value_counts(normalize=True).round(2)

In [ ]:
survival_by_status = titanic_df.groupby(by="Economic status")["Survived"].value_counts(normalize=True).round(2)
survival_by_status = survival_by_status.unstack().reset_index()
survival_by_status.index = survival_by_status["Economic status"]
survival_by_status = survival_by_status.drop(columns="Economic status")

In [ ]:
renamed_columns = {0:"Died",1:"Survived"}
survival_by_status = survival_by_status.rename(columns=renamed_columns)
survival_by_status.columns.name = None
survival_by_status

In [ ]:
plt.figure(figsize=(10, 5), dpi=100)
 
plt.subplot2grid(shape=(1,2),loc=(0,0))
squarify.plot(sizes=total_class_members,value=total_class_members, label=total_class_members.index, alpha=.8,color=['sienna', 'gold','lawngreen'])
plt.axis('off')  
plt.title("% of people according to economic status")   
    
plt.subplot2grid(shape=(1,2),loc=(0,1))
plt.bar(x=survival_by_status.index,height=survival_by_status["Survived"].values,color=['gold', 'lawngreen','sienna'])
plt.ylim(top=1)
plt.title("% of people that survived by class")

plt.show()

On the graphs we can see that people from lower class were more likely to die than from upper class.

### What happened to the families?

Here we will see if people with families had more chances of survival.

The dataset includes 2 attributes which count the number of members of each group.
* Number of siblings/spouses (sibsp):
    * Sibling = brother, sister, stepbrother, stepsister
    * Spouse = husband, wife (mistresses and fiancés were ignored)

* Number of parents/children (parch):
    * Parent = mother, father
    * Child = daughter, son, stepdaughter, stepson
    
*Note: Some children travelled only with a nanny, therefore parch=0 for them.*

In [ ]:
titanic_df["Number of siblings/spouses"].value_counts()

In [ ]:
titanic_df["Number of parents/children"].value_counts()

We can see that the majority of the passengers traveled alone.

In [ ]:
new_name = {"Number of siblings/spouses":"Total"}
sibsp = titanic_df.groupby(by="Number of siblings/spouses")["Survived"].value_counts(normalize=True).to_frame().unstack()
sibsp = sibsp.fillna(0)
sibsp.columns = sibsp.columns.get_level_values(1)
sibsp.columns.name = None
sibsp = sibsp.rename(columns={0:"Died",1:"Survived"}) 
sibsp

In [ ]:
bar_width = 0.3
start_first_bar = sibsp.index-bar_width/2
start_second_bar = sibsp.index+bar_width/2

plt.figure(figsize=(10, 4), dpi=100)
plt.bar(start_first_bar, sibsp["Died"], width = bar_width, color = 'lightcoral', edgecolor = 'black', label='Died')
plt.bar(start_second_bar, sibsp["Survived"], width = bar_width, color = 'lightgreen', edgecolor = 'black',  label='Survived')
plt.xticks(sibsp.index)
plt.ylabel('%')
plt.legend()
plt.title("Survival by number of siblings/spouses")
plt.show()

In the graph we can see that those with one or two siblings/spouses seem more likely to survive. Let's see now with the parents and children.

In [ ]:
new_name = {"Number of parents/children":"Total"}
parch = titanic_df.groupby(by="Number of parents/children")["Survived"].value_counts(normalize=True).to_frame().unstack()
parch = parch.fillna(0)
parch.columns = parch.columns.get_level_values(1)
parch.columns.name = None
parch = parch.rename(columns={0:"Died",1:"Survived"}) 
parch

In [ ]:
bar_width = 0.3
start_first_bar = parch.index-bar_width/2
start_second_bar = parch.index+bar_width/2

plt.figure(figsize=(10, 4), dpi=100)
plt.bar(start_first_bar, parch["Died"], width = bar_width, color = 'lightcoral', edgecolor = 'black', label='Died')
plt.bar(start_second_bar, parch["Survived"], width = bar_width, color = 'lightgreen', edgecolor = 'black',  label='Survived')
plt.xticks(parch.index)
plt.ylabel('%')
plt.legend()
plt.title("Survival by number of parents/children")
plt.show()

In this case we can see again that those with a family were more likely to survive than those that wet alone.

### Is there any relationship with the embarkation port?

In this section we will see if people that embarked in a certain port survived more than from the others.

In [ ]:
embark_ports = titanic_df["Embarked"].value_counts(normalize=True).round(2)

In [ ]:
survival_by_ports = titanic_df.groupby(by="Embarked")["Survived"].agg("mean")

In [ ]:
plt.figure(figsize=(10, 4), dpi=100)
plt.subplot2grid(shape=(1,2),loc=(0,0))
squarify.plot(sizes=embark_ports,value=embark_ports, label=embark_ports.index, alpha=.8,color=['darkslategrey', 'slategrey','cornflowerblue'])
plt.axis('off')  
plt.title("% of people from each port")   

plt.subplot2grid(shape=(1,2),loc=(0,1))
plt.bar(survival_by_ports.index, survival_by_ports.values, color = ['slategrey', 'cornflowerblue','darkslategrey'], alpha = 0.6)
plt.ylim(top=1)
plt.ylabel('%')
plt.title("Survival by port of embarkation")
plt.show()

We can see that those that embarked in Cherbourg had slightly better chances, though we have to take into account that Southhampton was the port from which most of the people embarked.

### Let's see the relationship between variables now

Now that we have seen each individual variable in its own, let's start to look for correlations between them.

In [ ]:
sns.lmplot( x='Age', y='Fare',data=titanic_df,fit_reg=False, hue='Survived', legend=True,height=7,aspect=1.5,markers=["x","o"],palette=['red','green'])
plt.title("Age/Fare with survival")
plt.show()

In this graph we can clearly see that those that paid a higher fare likely survived. Let's see if we can get a better view by making a graph for each sex.

In [ ]:
columns = ["Age","Fare","Survived"]
is_male = titanic_df["Sex"] == "Male"
fare_limit = titanic_df["Fare"] < 300 # So that we can see better the lower values
age_fare_male = titanic_df.loc[is_male & fare_limit,columns]
age_fare_female = titanic_df.loc[(~is_male) & fare_limit,columns]

In [ ]:
sns.lmplot( x='Age', y='Fare',data=age_fare_male,fit_reg=False, hue='Survived', legend=True,height=7,aspect=1.5,markers=["x","o"],palette=['red','green'])
plt.title("Age/Fare with survival, male")
plt.show()

In [ ]:
sns.lmplot( x='Age', y='Fare',data=age_fare_female,fit_reg=False, hue='Survived', legend=True,height=7,aspect=1.5,markers=["x","o"],palette=['red','green'])
plt.title("Age/Fare with survival, female")
plt.show()

This graph further confirms what we saw before, women more likely survived. In addition, we can see that also with the children.

Let's try to draw the boundaries of the socio-economic status by looking at different numerical values.

In [ ]:
titanic_df.groupby(by="Economic status")["Fare"].agg(["min","mean","max","std"]).round(2)

Seeing that the minimum fare was 0 for all economic status, I decided to take a look at these particular cases. 

In [ ]:
titanic_df.loc[titanic_df["Fare"] == 0]

By investigating a little, it turned out that these people boarded the ship without paying a fare for different reasons. For example, Andrews, Mr. Thomas Jr. was the naval architect for the ship, while others were part of the ["guarantee group"](https://www.encyclopedia-titanica.org/titanic-guarantee-group/). More info on [encyclopedia-titanica](https://www.encyclopedia-titanica.org/).


Let's continue without taking into account these cases.

In [ ]:
passengers_with_fare = titanic_df.loc[titanic_df["Fare"]>0]
passengers_with_fare.groupby(by="Economic status")["Fare"].agg(["min","mean","max","std"]).round(2)

By taking into account the mean and standard deviation, we can see that some groups overlap between each other in parts, thus blurring the boundaries. 

In [ ]:
# TODO: Sex/Class

## Conclusions
    
* Women were more likely to have survived.
* Many of the children survived.
* Families have better chances of survival.
* There are clear relationships between sex, gender, and socio-economic status.